
![title](https://ucm.es/themes/ucm16/media/img/logo.png)

# Bases de datos NoSQL

### Redis. Servicios de publicación/subscripción

Rafael Caballero

**Índice**

[Introducción](#Introducción)<br>
[Iniciar Servicio de publicación/subscripción](#Iniciar-Servicio-de-publicación/subscripción)<br>
[Publicación](#Publicación)<br>
[Subscripción](#Subscripción)<br>
[Estructura del mensaje](#Estructura-del-mensaje)<br>
[Patrones](#Patrones)<br>
[Modelos de lectura](#Modelos-de-lectura)<br>
[Enlaces de interés](#Enlaces-de-interés)<br>

## Introducción


Antes de nada conectamos a redis

In [1]:
# Ojo, cambiar estos datos por los de vuestro acceso a red en redislabs

redisconexion = "redis-13665.c55.eu-central-1-1.ec2.cloud.redislabs.com"
redispuerto = 13665
passwd = "csVe77ZtQL7sKQocZZHUlnjmSf0WpGxE"

import redis
r = redis.Redis(host=redisconexion, password=passwd, port=redispuerto, charset="utf-8", decode_responses=True, db=0)
r.ping() # debe mostrar True

True

Seguramente todos conocemos el patrón productor/consumidor, típico de programación concurrente: 

![image.png](https://miro.medium.com/max/1002/1*38NMAj0WTa_LD3ojoWsytQ.png)

En el patrón publicador/subscriptor el productor (aquí llamado publicador) se "independiza" del consumidor (aquí llamado subscriptor); se limita a publicar mensajes, normalmente en canales.

Análogemente los subscriptores se "apuntan" o "suscriben" a los canales que les interesan sin saber qué publicadores hay. 

Una diferencia importante es que cada suscriptor tiene su propio buzón, es decir cuando un suscriptor lee un mensaje no se lo quita a otro

![image.png](https://realtimeapi.io/wp-content/uploads/2017/09/pubsub-1.png)

## Iniciar Servicio de publicación/subscripción

In [9]:
p = r.pubsub()  
type(p)

redis.client.PubSub

## Publicación

Al publicar usamos la conexión a Redis y utilizamos el método publish

    publish(canal, valor)

donde tanto `canal` como `valor` son cadenas de caracteres.

Por ejemplo, supongamos que en una aplicación web tenemos dos canales para cada cliente conectado, uno con su carrito de la compra y otro con los datos de su sesión.

La siguiente información indica que Bertoldo ha comprado unos DVDs y herminia una TV. Finalmente también se publica que Bertoldo ha hecho logout.


In [10]:
r.publish('carrito-Bertoldo', 'DVDs Peppa Pig (edición del director)')
r.publish('carrito-Herminia', 'TV Sony, modelo Gordoten')
r.publish('sesion-Bertoldo','logout')

1

He hecho de utilizar el prefijo 'carrito' o 'sesión' nos permitirá aprovechar una utilidad de Redis: los patrones.

## Subscripción

Para suscribirnos basta con conocer el nombre del canal:

In [11]:
p.subscribe("sesion-Bertoldo")

Ahora podemos recibir mensajes:

In [12]:
getmsg = p.get_message()
print(getmsg)

{'type': 'subscribe', 'pattern': None, 'channel': 'sesion-Bertoldo', 'data': 1}


El mensaje:

    {'type': 'subscribe', 'pattern': None, 'channel': 'sesion-Bertoldo', 'data': 1}
    
indica que nos hemos suscrito con éxito. Al subscribirse siempre recibimos en primer lugar este mensaje de "bienvenida". Ahora ya podemos leer mensajes de verdad

In [13]:
getmsg = p.get_message()
print(getmsg)

None


¿Qué (censured) ha pasado?

Lo que ocurre es que hay que suscribirse antes de que se publiquen los mensajes, no a posteriori

In [24]:
s1 = r.pubsub()  
s2 = r.pubsub()
s3 = r.pubsub()
s1.subscribe("sesion-Bertoldo")
s2.subscribe("sesion-Bertoldo")

#### aquí llega el publisher...esto sería otra aplicación

r.publish('sesion-Bertoldo','logout')

# un despistado
s3.subscribe("sesion-Bertoldo")


### ahora a recibir mensajes

print("Subscriptor 1")
getmsg = s1.get_message()
print(getmsg)
getmsg = s1.get_message()
print(getmsg)

print("Subscriptor 2")
getmsg = s2.get_message()
print(getmsg)
getmsg = s2.get_message()
print(getmsg)

print("Subscriptor 3")
getmsg = s3.get_message()
print(getmsg)
getmsg = s3.get_message()
print(getmsg)


# eliminamos las subscripciones
s1.unsubscribe()
s2.unsubscribe()
s3.unsubscribe()


Subscriptor 1
{'type': 'subscribe', 'pattern': None, 'channel': 'sesion-Bertoldo', 'data': 1}
{'type': 'message', 'pattern': None, 'channel': 'sesion-Bertoldo', 'data': 'logout'}
Subscriptor 2
{'type': 'subscribe', 'pattern': None, 'channel': 'sesion-Bertoldo', 'data': 1}
{'type': 'message', 'pattern': None, 'channel': 'sesion-Bertoldo', 'data': 'logout'}
Subscriptor 3
None
None


## Estructura del mensaje

El método `get_message` devuelve un diccionario Python con la siguiente estructura:

* `Type`: Tipo del mensaje. Seis tipos: ‘subscribe’, ‘unsubscribe’, ‘psubscribe’, ‘punsubscribe’, ‘message’, ‘pmessage’

* `Pattern`: Si encaja con algún patrón (ver más adelante)

* `Channel`: Nombre del canal.

* `Data`: Si el tipo es `pmessage`entonces contiene el mensaje.

Ejemplo:

In [37]:
s1 = r.pubsub()  
s2 = r.pubsub()
s1.subscribe("c1")
s1.subscribe("c2")
s2.subscribe("c2")

#### aquí llega el publisher...esto sería otra aplicación

r.publish('c1',1)
r.publish('c2',2)



3

In [38]:
### ahora a recibir mensajes

print("Subscriptor 1")
getmsg = s1.get_message()
print(getmsg)
getmsg = s1.get_message()
print(getmsg)
getmsg = s1.get_message() # qué escribirá este en concreto?
print(getmsg)

print("Subscriptor 2")
getmsg = s2.get_message()
print(getmsg)
getmsg = s2.get_message()
print(getmsg)
getmsg = s2.get_message() 
print(getmsg)

# eliminamos las subscripciones
s1.unsubscribe()
s2.unsubscribe()

Subscriptor 1
{'type': 'subscribe', 'pattern': None, 'channel': 'c1', 'data': 1}
{'type': 'subscribe', 'pattern': None, 'channel': 'c2', 'data': 2}
{'type': 'message', 'pattern': None, 'channel': 'c1', 'data': '1'}
Subscriptor 2
{'type': 'subscribe', 'pattern': None, 'channel': 'c2', 'data': 1}
{'type': 'message', 'pattern': None, 'channel': 'c2', 'data': '2'}
None


## Patrones

También se pueden utilizar patrones para suscribirse a varios canales a la vez. 

Supongamos que cada usuario tiene su propio carrito de la compra con el nombre `carrito-nombre`y queremos suscribirnos a todos a la vez


In [39]:
carritos = r.pubsub()  
carritos.psubscribe('carrito-*')

r.publish('carrito-Bertoldo', 'DVDs Peppa Pig (edición del director)')
r.publish('carrito-Herminia', 'TV Sony, modelo Gordoten')

seguir = True
while seguir:
    getmsg = carritos.get_message()
    print(getmsg)
    seguir = getmsg != None

# eliminamos las subscripciones
carritos.unsubscribe()

{'type': 'psubscribe', 'pattern': None, 'channel': 'carrito-*', 'data': 1}
{'type': 'pmessage', 'pattern': 'carrito-*', 'channel': 'carrito-Bertoldo', 'data': 'DVDs Peppa Pig (edición del director)'}
{'type': 'pmessage', 'pattern': 'carrito-*', 'channel': 'carrito-Herminia', 'data': 'TV Sony, modelo Gordoten'}
None


## Modelos de lectura

Si se espera un flujo de mensajes, el subscriptor tiene varias posibilidades.

La primera es utilizar `get_message` que, como hemos visto, es asíncrono:

In [49]:
import time
## preparamos la subscripción
raton = r.pubsub()
raton.subscribe("raton")

# aquí se publican los mensajes, esto iría en otra aplicación
print("Moviendo ratón")
for i in range(50):
    r.publish('raton', 'arriba')
for i in range(100):
    r.publish('raton', 'derecha')
r.publish('raton', 'queso!')


# coordenadas de un imaginario ratón
print("Leyendo movimiento ratonil")
x=0
y=0
seguir = True
while seguir:
    message = raton.get_message()
    if message:
        datos = message["data"]
        #print(datos)
        if datos=="arriba":
            y+=1
        elif datos=="abajo":
            y-=1
        elif datos=="derecha": 
            x+=1
        elif datos=="izquierda": 
            x-=1 
        elif datos=="queso!": 
            seguir = False
    else:
        # no hay mensajes, podemos ocuparnos de otros asuntos
        time.sleep(0.01) 

print(x,y)
# eliminamos las subscripciones
raton.unsubscribe()

Moviendo ratón
Leyendo movimiento ratonil
100 50


In [46]:
ordenes.unsubscribe()

## Enlaces de interés

* Para entender mejor el problema del productor consumidor en el contexto de la programación concurrente podéis ver este [vídeo](https://www.youtube.com/watch?v=LRiN3DJdskA)<br>

* [Información de Redis sobre publicación subscripción](https://redis.io/topics/pubsub) <br>

* La [documentación de redis-py](https://github.com/andymccurdy/redis-py#publish--subscribe) tiene información detallada sbbre su uso desde Python<br>


* Para publicación subscripción hay otras aplicaciones que os pueden interesar. Se las suele denominar "message brokers" y entre otras podéis mirar [Apache Kafka](https://kafka.apache.org/) para entornos Hadoop / big data, [RabbitMQ](https://www.rabbitmq.com/). [Aquí](https://www.cloudamqp.com/blog/2019-12-12-when-to-use-rabbitmq-or-apache-kafka.html) tenéis una comparación entre ambos. En este [otro enlace](https://blog.scottlogic.com/2018/04/17/comparing-big-data-messaging.html) presentan otras alternativas